# Introduction - Titanic Linear Regression

Name: Gabriel Richards / gjrich

Date: 5 Apr 2025

This lab uses the Titanic dataset to review regression modeling techniques. Previous work focused on classification (predicting survival), but we'll now predict the fare passengers paid for their journey. We'll review how  passenger attributes like age, family size, and other features correlate with ticket pricing. We'll also build multiple regression models including Linear Regression, Ridge, Elastic Net, and Polynomial Regression, and evaluate their performance using R², RMSE, and MAE to determine which features and models best predict passenger fares.

## Python Package Imports

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import root_mean_squared_error


## Section 1. Import and Inspect the Data

Load the Titanic dataset and confirm it’s structured correctly.


Important: This code requires importing seaborn as sns and pandas. Our variable titanic holds a pandas DataFrame object. Know what imports are required for each bit of code. 

In [ ]:
# Load Titanic dataset from seaborn and verify
titanic = sns.load_dataset("titanic")
titanic.head(20)

In [ ]:
titanic.info()

In [ ]:
# Check for null/NaN values
null_count = titanic['embark_town'].isna().sum()
print(f"Number of null values in embark_town: {null_count}")

# Check for empty strings
empty_string_count = (titanic['embark_town'] == '').sum()
print(f"Number of empty strings in embark_town: {empty_string_count}")

# Check for whitespace-only strings
whitespace_count = (titanic['embark_town'].str.isspace()).sum()
print(f"Number of whitespace-only strings in embark_town: {whitespace_count}")

# Show rows with missing embark_town
missing_embark = titanic[titanic['embark_town'].isna()]
print("\nRows with missing embark_town:")
print(missing_embark[['survived', 'pclass', 'sex', 'age', 'fare']])

## Section 2. Data Exploration and Preparation

Prepare the Titanic data for regression modeling. See the previous work.

- Impute missing values for age using median
- Drop rows with missing fare (or impute if preferred)
- Create numeric variables (e.g., family_size from sibsp + parch + 1)
- Optional - convert categorical features (e.g. sex, embarked) if you think they might help your prediction model. (We do not know relationships until we evaluate things.)

In [ ]:
# Calculate the mean and standard deviation of known ages
age_mean = titanic['age'].mean()
age_std = titanic['age'].std()

# Count missing values
age_null_count = titanic['age'].isnull().sum()

# Generate random ages from normal distribution 
np.random.seed(307)  # For reproducibility
age_random_values = np.random.normal(age_mean, age_std, age_null_count)

# Apply age boundary constraints (0 to 85 years)
age_random_values = np.clip(age_random_values, 0, 85)

# Round ages to whole numbers
age_random_values = np.round(age_random_values)

# Create a mask for rows with missing age
age_null_mask = titanic['age'].isnull()

# Fill missing values with the random ages
titanic.loc[age_null_mask, 'age'] = age_random_values

# Round all ages in the dataset for consistency
titanic['age'] = titanic['age'].round()

# Verify no missing age values remain
print(f"Missing age values after imputation: {titanic['age'].isnull().sum()}")

titanic = titanic.dropna(subset=['fare'])

titanic['family_size'] = titanic['sibsp'] + titanic['parch'] + 1

## Section 3. Feature Selection and Justification
Define multiple combinations of features to use as inputs to predict fare.

Use unique names (X1, y1, X2, y2, etc.) so results are visible and can be compared at the same time. 

Remember the inputs, usually X, are a 2D array. The target is a 1D array. 

In [ ]:
# Case 1. age
X1 = titanic[['age']]
y1 = titanic['fare']

In [ ]:
# Case 2. family_size
X2 = titanic[['family_size']]
y2 = titanic['fare']

In [ ]:
# Case 3. age, family_size
X3 = titanic[['age', 'family_size']]
y3 = titanic['fare']

In [ ]:
# Case 4. Embark_town

# Converting embark_town and who to numeric values
# First, make a copy to avoid warnings about modifying the DataFrame
titanic_prep = titanic.copy()

# Convert embark_town to numeric
titanic_prep['embark_town_numeric'] = titanic_prep['embark_town'].map({
    'Southampton': 0, 
    'Cherbourg': 1, 
    'Queenstown': 2
})

# Fill any NaN values that might result from the mapping
titanic_prep['embark_town_numeric'] = titanic_prep['embark_town_numeric'].fillna(0)  # Default to Southampton
# Case 4. embark_town and who (converted to numeric)

X4 = titanic_prep[['embark_town_numeric']]
y4 = titanic_prep['fare']

In [ ]:
# Case 5. pclass
X5 = titanic[['pclass']]
y5 = titanic['fare']

### Reflection Questions:

#### Why might these features affect a passenger's fare:
- Embarkation town could affect fare due to varying pricing at each port 
- Additionally, certain ports might have catered to wealthier clientbases

#### List all available features:
- survived, pclass, sex, age, sibsp, parch, fare, embarked, class, who, adult_male, deck, embark_town, alive, alone

#### Which other features could improve predictions and why:
Pclass would likely have the strongest effect as ticket class was the primary fare determinant. Deck could indicate premium accommodation pricing. Family size (sibsp + parch) might reveal group pricing patterns. Age could show different pricing tiers for adults vs. children. Sex might indicate if there were gender-based pricing differences in that era.

#### How many variables are in your Case 4:
One variable: embark_town_numeric

#### Which variable(s) did you choose for Case 4 and why do you feel those could make good inputs:
I chose embark_town because geographic location could significantly impact ticket pricing. Each port had different market conditions, operating costs, and possibly different pricing strategies. Queenstown, Southampton, and Cherbourg likely had different base fare structures. This variable allows the model to detect if location-based pricing was a significant factor independent of passenger class or demographic characteristics.

## Section 4. Train a Regression Model (Linear Regression)


### 4.1 Split the Data


In [ ]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=123)

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=123)

X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.2, random_state=123)

X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, test_size=0.2, random_state=123)

X5_train, X5_test, y5_train, y5_test = train_test_split(X5, y5, test_size=0.2, random_state=123)

### 4.2 Train and Evaluate Linear Regression Models (all 4 cases)


We'll use a more concise approach - create each model and immediately call the fit() method. 

In [ ]:
lr_model1 = LinearRegression().fit(X1_train, y1_train)
lr_model2 = LinearRegression().fit(X2_train, y2_train)
lr_model3 = LinearRegression().fit(X3_train, y3_train)
lr_model4 = LinearRegression().fit(X4_train, y4_train)
lr_model5 = LinearRegression().fit(X5_train, y5_train)


# Predictions

y_pred_train1 = lr_model1.predict(X1_train)
y_pred_test1 = lr_model1.predict(X1_test)

y_pred_train2 = lr_model2.predict(X2_train)
y_pred_test2 = lr_model2.predict(X2_test)

y_pred_train3 = lr_model3.predict(X3_train)
y_pred_test3 = lr_model3.predict(X3_test)

y_pred_train4 = lr_model4.predict(X4_train)
y_pred_test4 = lr_model4.predict(X4_test)

y_pred_train5 = lr_model5.predict(X5_train)
y_pred_test5 = lr_model5.predict(X5_test)

### 4.3 Report Performance


In [ ]:
# Case 1 Performance
print("Case 1: Training R²:", r2_score(y1_train, y_pred_train1))
print("Case 1: Test R²:", r2_score(y1_test, y_pred_test1))
print("Case 1: Test RMSE:", root_mean_squared_error(y1_test, y_pred_test1))
print("Case 1: Test MAE:", mean_absolute_error(y1_test, y_pred_test1))

# Case 2 Performance
print("Case 2: Training R²:", r2_score(y2_train, y_pred_train2))
print("Case 2: Test R²:", r2_score(y2_test, y_pred_test2))
print("Case 2: Test RMSE:", root_mean_squared_error(y2_test, y_pred_test2))
print("Case 2: Test MAE:", mean_absolute_error(y2_test, y_pred_test2))

# Case 3 Performance
print("Case 3: Training R²:", r2_score(y3_train, y_pred_train3))
print("Case 3: Test R²:", r2_score(y3_test, y_pred_test3))
print("Case 3: Test RMSE:", root_mean_squared_error(y3_test, y_pred_test3))
print("Case 3: Test MAE:", mean_absolute_error(y3_test, y_pred_test3))

# Case 4 Performance
print("Case 4: Training R²:", r2_score(y4_train, y_pred_train4))
print("Case 4: Test R²:", r2_score(y4_test, y_pred_test4))
print("Case 4: Test RMSE:", root_mean_squared_error(y4_test, y_pred_test4))
print("Case 4: Test MAE:", mean_absolute_error(y4_test, y_pred_test4))

# Case 5 Performance
print("Case 5: Training R²:", r2_score(y5_train, y_pred_train5))
print("Case 5: Test R²:", r2_score(y5_test, y_pred_test5))
print("Case 5: Test RMSE:", root_mean_squared_error(y5_test, y_pred_test5))
print("Case 5: Test MAE:", mean_absolute_error(y5_test, y_pred_test5))

### Section 4 Reflection Questions

#### Compare the train vs test results for each.

While I did answer the questions for cases 1-4, it's worth saying that with extremely low R² values, the concepts of overfitting and underfitting become somewhat meaningless. For a model to meaningfully overfit or underfit, there should be an actual pattern in the data that can be captured. When R² values are near zero (like 0.005 or 0.05), they indicate that essentially no relationship exists between the predictors and the target variable.

1. **Did Case 1 overfit or underfit? Explain:**
   Age -  underfit. Both training R² (0.0061) and test R² (0.0055) are extremely low and similar, indicating the model failed to capture meaningful patterns between age and fare.

2. **Did Case 2 overfit or underfit? Explain:**
   family_size - showed slight overfitting. The training R² (0.0499) is more than double the test R² (0.0222), indicating the model learned patterns in training data that didn't generalize well to test data. Still, both values are very low.

3. **Did Case 3 overfit or underfit? Explain:**
   age and family_size - showed slight overfitting as the training R² (0.0662) is higher than the test R² (0.0487). This captured more information than the first two cases, but still explains less than 5% of fare variance.

4. **Did Case 4 overfit or underfit? Explain:**
   embark_town - likely underfit, though it shows unusual behavior with test R² (0.0085) higher than training R² (0.0031). Both values are extremely low, indicating the model failed to capture meaningful patterns. The reversed R² values suggest random variation in the data split rather than a true pattern.

5. **Did Case 5 overfit or underfit? Explain:**
   pclass - showed balanced fitting. I included it as a bonus sanity test since the correlations were so low on the other cases. The training R² (0.301) and test R² (0.302) are virtually identical, indicating good generalization. This model explains about 30% of fare variance, significantly better than all other cases.


#### Adding Age

1. **Did adding age improve the model:**
   Yes, infinitesmally. Comparing Case 2 (family_size only, R²=0.022) to Case 3 (age and family_size, R²=0.049), adding age did double the test R² and reduced MAE from 25.03 to 24.13...but despite that, the correlation generally remained very low.

2. **Propose a possible explanation:**
   Age might effect ticket price because: (1) different fare structures may have existed for children vs. adults, (2) age may correlate with wealth or social status that influenced cabin class selection, or (3) families with young children might have required different accommodations. However, the low R² values suggest this relationship is still quite weak compared to other factors like passenger class.


#### Worst

1. **Which case performed the worst:**
   Case 1 - age

2. **How do you know:**
   It has the lowest test R² (0.0055), and the highest RMSE [37.93].

3. **Do you think adding more training data would improve it (and why/why not):**
   No. The problem isn't insufficient data, but that age alone has almost no relationship with fare (R² near zero). More data with the same weak predictor probably wouldn't improve performance.
   

#### Best

1. **Which case performed the best:**
   Case 5  - pclass

2. **How do you know:**
   It has much higher test R² (0.302) than the others. It also has the lowest RMSE (31.79) and MAE (20.65), indicating more accurate predictions.

3. **Do you think adding more training data would improve it (and why/why not):**
   No, I think not. The model already shows consistent generalization, nearly identical between train and test R². However adding different features like deck or combining with other features might likely yield better improvements than more training data.

## Section 5. Compare Alternative Models
In this section, we will take the best-performing case and explore other regression models.

Choose Best Case to Continue
Choose the best case model from the four cases. Use that model to continue to explore additional continuous prediction models. The following assumes that Case 1 was the best predictor  - this may not be the case. Adjust the code to use your best case model instead. 

Choosing Options
When working with regression models, especially those with multiple input features, we may run into overfitting — where a model fits the training data too closely and performs poorly on new data. To prevent this, we can apply regularization.

Regularization adds a penalty to the model’s loss function, discouraging it from using very large weights (coefficients). This makes the model simpler and more likely to generalize well to new data.

In general: 

If the basic linear regression is overfitting, try Ridge.

If you want the model to automatically select the most important features, try Lasso.

If you want a balanced approach, try Elastic Net.

### 5.1 Ridge Regression (L2 penalty)
Ridge Regression is a regularized version of linear regression that adds a penalty to large coefficient values. It uses the L2 penalty, which adds the sum of squared coefficients to the loss function.

This "shrinks" the coefficients, reducing the model’s sensitivity to any one feature while still keeping all features in the model.

Penalty term: L2 = sum of squared weights
Effect: Shrinks weights, helps reduce overfitting, keeps all features

In [ ]:
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X1_train, y1_train)
y_pred_ridge = ridge_model.predict(X1_test)

### 5.2 Elastic Net (L1 + L2 combined)
Lasso Regression uses the L1 penalty, which adds the sum of absolute values of the coefficients to the loss function. Lasso can shrink some coefficients all the way to zero, effectively removing less important features. This makes it useful for feature selection.

Penalty term: L1 = sum of absolute values of weights
Effect: Can shrink some weights to zero (drops features), simplifies the model
Elastic Net combines both L1 (Lasso) and L2 (Ridge) penalties. It balances the feature selection ability of Lasso with the stability of Ridge.

We control the balance with a parameter called l1_ratio:

If l1_ratio = 0, it behaves like Ridge
If l1_ratio = 1, it behaves like Lasso
Values in between mix both types
Penalty term: α × (L1 + L2)
Effect: Shrinks weights and can drop some features — flexible and powerful

In [ ]:
elastic_model = ElasticNet(alpha=0.3, l1_ratio=0.5)
elastic_model.fit(X1_train, y1_train)
y_pred_elastic = elastic_model.predict(X1_test)

### 5.3 Polynomial Regression
Linear regression is a simple two dimensional relationship - a simple straight line. But we can test more complex relationships. Polynomial regression adds interaction and nonlinear terms to the model. Be careful here - higher-degree polynomials can easily overfit.

In [ ]:
# Set up the poly inputs
poly = PolynomialFeatures(degree=3)
X_train_poly = poly.fit_transform(X1_train)
X1_test_poly = poly.transform(X1_test)
 

# Use the poly inputs in the LR model
poly_model = LinearRegression()
poly_model.fit(X_train_poly, y1_train)
y_pred_poly = poly_model.predict(X1_test_poly)

### 5.4 Visualize Polynomial Cubic Fit (for 1 input feature)
Choose a case with just one input feature and plot it. For example:

In [ ]:
plt.scatter(X1_test, y1_test, color='blue', label='Actual')
plt.scatter(X1_test, y_pred_poly, color='red', label='Predicted (Poly)')
plt.legend()
plt.title("Polynomial Regression: Age vs Fare")
plt.show()

In [ ]:
def report(name, y_true, y_pred):
    print(f"{name} R²: {r2_score(y_true, y_pred):.3f}")
    print(f"{name} RMSE: {root_mean_squared_error(y_true, y_pred):.2f}")
    print(f"{name} MAE: {mean_absolute_error(y_true, y_pred):.2f}\n")

report("Linear", y1_test, y_pred_test1)
report("Ridge", y1_test, y_pred_ridge)
report("ElasticNet", y1_test, y_pred_elastic)
report("Polynomial", y1_test, y_pred_poly)

## 5.5 Visualize Higher Order Polynomial (for the same 1 input case)

Use the same single input case as you visualized above, but use a higher degree polynomial (e.g. 4, 5, 6, 7, or 8). Plot the result. 

In a Markdown cell, tell us which option seems to work better - your initial cubic (3) or your higher order and why. 

In [ ]:
# Testing multiple polynomial degrees
degrees = [3, 4, 6, 8]
poly_models = {}
y_preds = {}
r2_scores = {}

# Train models for each degree
for degree in degrees:
    # Create polynomial features
    poly_transformer = PolynomialFeatures(degree=degree)
    X_train_poly_deg = poly_transformer.fit_transform(X1_train)
    X_test_poly_deg = poly_transformer.transform(X1_test)
    
    # Train model
    model = LinearRegression()
    model.fit(X_train_poly_deg, y1_train)
    
    # Generate predictions
    y_pred = model.predict(X_test_poly_deg)
    
    # Store results
    poly_models[degree] = model
    y_preds[degree] = y_pred
    r2_scores[degree] = r2_score(y1_test, y_pred)

# Create plots for each degree
plt.figure(figsize=(15, 10))

for i, degree in enumerate(degrees, 1):
    plt.subplot(2, 2, i)
    plt.scatter(X1_test, y1_test, color='blue', alpha=0.6, label='Actual')
    plt.scatter(X1_test, y_preds[degree], color=f'C{i}', alpha=0.6, label=f'Degree {degree}')
    
    # To visualize the curve better, let's sort the points by x-value
    sorted_indices = X1_test.iloc[:, 0].argsort()
    sorted_X = X1_test.iloc[sorted_indices]
    sorted_y_pred = y_preds[degree][sorted_indices]
    
    # Draw a line connecting predictions
    plt.plot(sorted_X, sorted_y_pred, color=f'C{i}', linestyle='-', alpha=0.7)
    
    plt.title(f"Polynomial Regression (Degree {degree}): Age vs Fare")
    plt.xlabel("Age")
    plt.ylabel("Fare")
    plt.legend()
    
    # Add R² value to plot
    plt.annotate(f"R² = {r2_scores[degree]:.4f}", 
                 xy=(0.05, 0.95), xycoords='axes fraction',
                 bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))

plt.tight_layout()
plt.show()

# Report performance metrics for each model
print("Performance Metrics for Different Polynomial Degrees:\n")
for degree in degrees:
    print(f"Polynomial Degree {degree}:")
    print(f"R²: {r2_scores[degree]:.4f}")
    print(f"RMSE: {root_mean_squared_error(y1_test, y_preds[degree]):.2f}")
    print(f"MAE: {mean_absolute_error(y1_test, y_preds[degree]):.2f}")
    print("-" * 40)

In [ ]:
# Testing multiple polynomial degrees using pclass as the input feature
degrees = [3, 4, 6, 8]
poly_models_pclass = {}
y_preds_pclass = {}
r2_scores_pclass = {}

# Train models for each degree
for degree in degrees:
    # Create polynomial features
    poly_transformer = PolynomialFeatures(degree=degree)
    X_train_poly_deg = poly_transformer.fit_transform(X5_train)  # Using X5 (pclass) instead of X1 (age)
    X_test_poly_deg = poly_transformer.transform(X5_test)
    
    # Train model
    model = LinearRegression()
    model.fit(X_train_poly_deg, y5_train)
    
    # Generate predictions
    y_pred = model.predict(X_test_poly_deg)
    
    # Store results
    poly_models_pclass[degree] = model
    y_preds_pclass[degree] = y_pred
    r2_scores_pclass[degree] = r2_score(y5_test, y_pred)

# Create plots for each degree
plt.figure(figsize=(15, 10))

for i, degree in enumerate(degrees, 1):
    plt.subplot(2, 2, i)
    plt.scatter(X5_test, y5_test, color='blue', alpha=0.6, label='Actual')
    plt.scatter(X5_test, y_preds_pclass[degree], color=f'C{i}', alpha=0.6, label=f'Degree {degree}')
    
    # Create sorted data for a cleaner line plot
    sorted_indices = X5_test.iloc[:, 0].argsort()
    sorted_X = X5_test.iloc[sorted_indices]
    sorted_y_pred = y_preds_pclass[degree][sorted_indices]
    
    # Draw a line connecting predictions
    plt.plot(sorted_X, sorted_y_pred, color=f'C{i}', linestyle='-', alpha=0.7)
    
    plt.title(f"Polynomial Regression (Degree {degree}): Pclass vs Fare")
    plt.xlabel("Passenger Class")
    plt.ylabel("Fare")
    plt.legend()
    plt.xticks([1, 2, 3], ['First', 'Second', 'Third'])
    
    # Add R² value to plot
    plt.annotate(f"R² = {r2_scores_pclass[degree]:.4f}", 
                 xy=(0.05, 0.95), xycoords='axes fraction',
                 bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))

plt.tight_layout()
plt.show()

# Report performance metrics for each model
print("Performance Metrics for Different Polynomial Degrees (Pclass):\n")
for degree in degrees:
    print(f"Polynomial Degree {degree}:")
    print(f"R²: {r2_scores_pclass[degree]:.4f}")
    print(f"RMSE: {root_mean_squared_error(y5_test, y_preds_pclass[degree]):.2f}")
    print(f"MAE: {mean_absolute_error(y5_test, y_preds_pclass[degree]):.2f}")
    print("-" * 40)

R^2 values are good compared to other cases, but clearly changing the degree makes no difference and the visualizations are unclear. Let's try combining with P-class

In [ ]:
# Create a combined feature set with pclass and family_size
X_combined = titanic[['pclass', 'family_size']]

# Split the data
X_combined_train, X_combined_test, y_combined_train, y_combined_test = train_test_split(
    X_combined, titanic['fare'], test_size=0.2, random_state=123)

# Train a linear regression model
lr_combined = LinearRegression()
lr_combined.fit(X_combined_train, y_combined_train)
y_pred_combined = lr_combined.predict(X_combined_test)

# Create a 3D visualization of the multi-feature regression
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot the actual data points
ax.scatter(X_combined_test['pclass'], X_combined_test['family_size'], y_combined_test, 
           color='blue', alpha=0.6, marker='o', s=40, label='Actual')

# Plot predicted data points
ax.scatter(X_combined_test['pclass'], X_combined_test['family_size'], y_pred_combined, 
           color='red', alpha=0.6, marker='^', s=40, label='Predicted')

# Set labels and title
ax.set_xlabel('Passenger Class')
ax.set_ylabel('Family Size')
ax.set_zlabel('Fare')
ax.set_title('Linear Regression: Pclass & Family Size vs Fare')
ax.set_xticks([1, 2, 3])
ax.set_xticklabels(['First', 'Second', 'Third'])

# Set a lower viewing angle
ax.view_init(elev=20, azim=-35)  # Lower elevation, adjust azimuth

# Add R² value to plot
ax.text2D(0.05, 0.95, f"R² = {r2_score(y_combined_test, y_pred_combined):.4f}", 
         transform=ax.transAxes,
         bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))

# Make legend more visible
plt.legend(loc='upper right', bbox_to_anchor=(1, 0.9), fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# Testing multiple polynomial degrees with combined features
degrees = [3, 4, 6, 8]
poly_models_combined = {}
y_preds_combined = {}
r2_scores_combined = {}

# Define distinct colors for better contrast
colors = ['orange', 'green', 'purple', 'red']

# Train models for each degree
for degree in degrees:
    # Create polynomial features
    poly_transformer = PolynomialFeatures(degree=degree)
    X_train_poly_deg = poly_transformer.fit_transform(X_combined_train)
    X_test_poly_deg = poly_transformer.transform(X_combined_test)
    
    # Train model
    model = LinearRegression()
    model.fit(X_train_poly_deg, y_combined_train)
    
    # Generate predictions
    y_pred = model.predict(X_test_poly_deg)
    
    # Store results
    poly_models_combined[degree] = model
    y_preds_combined[degree] = y_pred
    r2_scores_combined[degree] = r2_score(y_combined_test, y_pred)

# Create plots for each degree using 3D visualization
plt.figure(figsize=(16, 12))

for i, degree in enumerate(degrees):
    # Create 3D subplot
    ax = plt.subplot(2, 2, i+1, projection='3d')
    
    # Plot actual data points
    ax.scatter(X_combined_test['pclass'], X_combined_test['family_size'], y_combined_test, 
               color='blue', alpha=0.5, marker='o', s=30, label='Actual')
    
    # Plot predicted data points
    ax.scatter(X_combined_test['pclass'], X_combined_test['family_size'], y_preds_combined[degree], 
               color=colors[i], alpha=0.7, marker='^', s=40, label=f'Degree {degree}')
    
    # Set labels and title
    ax.set_xlabel('Passenger Class')
    ax.set_ylabel('Family Size')
    ax.set_zlabel('Fare')
    ax.set_title(f"Polynomial (Degree {degree}): Pclass & Family Size vs Fare")
    ax.set_xticks([1, 2, 3])
    ax.set_xticklabels(['First', 'Second', 'Third'])
    
    # Set a lower viewing angle
    ax.view_init(elev=20, azim=-35)  # Lower elevation, adjust azimuth
    
    # Add R² value to plot
    ax.text2D(0.05, 0.95, f"R² = {r2_scores_combined[degree]:.4f}", 
             transform=ax.transAxes,
             bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))
    
    # Add legend
    ax.legend(loc='upper right', fontsize=10)

plt.tight_layout()
plt.subplots_adjust(wspace=0.15, hspace=0.2)  # Adjust spacing
plt.show()

# Report performance metrics for each model
print("Performance Metrics for Different Polynomial Degrees (Combined Features):\n")
for i, degree in enumerate(degrees):
    print(f"Polynomial Degree {degree} ({colors[i]}):")
    print(f"R²: {r2_scores_combined[degree]:.4f}")
    print(f"RMSE: {root_mean_squared_error(y_combined_test, y_preds_combined[degree]):.2f}")
    print(f"MAE: {mean_absolute_error(y_combined_test, y_preds_combined[degree]):.2f}")
    print("-" * 40)

Yes, the decreasing R² value as the polynomial degree increases (from degree 4 to degrees 6 and 8) is indeed a classic sign of overfitting. Here's what's happening:

With degree 4 (R² = 0.5187), the model has found a good balance between complexity and generalization. It captures the underlying patterns in the data without fitting to noise.
As we increase to degree 6 (R² = 0.5025), the model becomes more complex but actually performs worse on test data. This suggests it's starting to fit to random fluctuations in the training data.
By degree 8 (R² = 0.4912), the overfitting is even more pronounced, with further decreased performance.

This pattern is a textbook example of the bias-variance tradeoff in machine learning:

Too simple (low degree): high bias, underfitting
Good balance (degree 4): captures real patterns
Too complex (higher degrees): high variance, overfitting

This is why it's generally good practice to try multiple model complexities and select the one that performs best on validation/test data rather than just assuming that more complexity will always yield better results.

Section 6. Final Thoughts & Insights
Your notebook should tell a data story. Use this section to demonstrate your thinking and value as an analyst.

6.1 Summarize Findings
What features were most useful?

What regression model performed best?

How did model complexity or regularization affect results?

 

6.2 Discuss Challenges
Was fare hard to predict? Why?

Did skew or outliers impact the models?

 

6.3 Optional Next Steps
Try different features besides the ones used (e.g., pclass, sex if you didn't use them this time)

Try predicting age instead of fare

Explore log transformation of fare to reduce skew

 